# Bronze to Silver - PII (Dados Sensíveis)

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import (
    col, trim, upper, lower, regexp_replace, initcap,
    when, coalesce, lit, current_timestamp, row_number, length
)
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from datetime import datetime

## Configurações

In [0]:
# Paths
STORAGE_ACCOUNT = "mystoacc"
BRONZE_PATH = f"abfss://bronze@{STORAGE_ACCOUNT}.dfs.core.windows.net/pii"
SILVER_PATH = f"abfss://silver@{STORAGE_ACCOUNT}.dfs.core.windows.net/pii"

# Catalog e Database
spark.sql("USE CATALOG hive_metastore")
spark.sql("USE healthcare_silver")

# Timestamp de processamento
PROCESSING_TIMESTAMP = datetime.now()
PROCESSING_DATE = PROCESSING_TIMESTAMP.strftime("%Y-%m-%d")

print(f"Processamento PII iniciado: {PROCESSING_TIMESTAMP}")
print("ATENÇÃO: Dados sensíveis - LGPD")


## Funções de Qualidade

In [0]:
def add_silver_metadata(df: DataFrame) -> DataFrame:
    """Adiciona colunas de metadados da Silver"""
    return df.withColumn("silver_processed_at", lit(PROCESSING_TIMESTAMP)) \
             .withColumn("silver_processing_date", lit(PROCESSING_DATE))

def remove_duplicates(df: DataFrame, key_column: str) -> DataFrame:
    """Remove duplicatas mantendo o registro mais recente"""
    window_spec = Window.partitionBy(key_column).orderBy(col("ingestion_timestamp").desc())
    return df.withColumn("row_num", row_number().over(window_spec)) \
             .filter(col("row_num") == 1) \
             .drop("row_num")

def clean_cpf(cpf_col: str) -> col:
    """Remove caracteres não numéricos do CPF"""
    return regexp_replace(col(cpf_col), r"[^\d]", "")

def clean_phone(phone_col: str) -> col:
    """Remove caracteres não numéricos do telefone"""
    return regexp_replace(col(phone_col), r"[^\d]", "")

def validate_cpf_length(cpf_col: str) -> col:
    """Valida se CPF tem 11 dígitos"""
    return when(length(col(cpf_col)) == 11, col(cpf_col)).otherwise(None)


## Transformações PII

In [0]:
def transform_paciente_identidade(df: DataFrame) -> DataFrame:
    """
    Transformações paciente_identidade:
    - Validar sk_paciente (obrigatório)
    - Validar CPF (11 dígitos)
    - Padronizar nome (initcap)
    - Validar email (básico)
    """
    df_transformed = df.select(
        col("sk_paciente").cast("int"),
        col("cpf").cast("string").alias("cpf_raw"),
        initcap(trim(col("nome_completo"))).alias("nome_completo"),
        lower(trim(col("email"))).alias("email"),
        col("ingestion_timestamp"),
        col("is_pii")
    )
    
    # Validar CPF (11 dígitos quando convertido pra string)
    df_transformed = df_transformed.withColumn(
        "cpf",
        when(length(col("cpf_raw")) == 11, col("cpf_raw")).otherwise(None)
    ).drop("cpf_raw")
    
    # sk_paciente e CPF são obrigatórios
    df_transformed = df_transformed.filter(
        col("sk_paciente").isNotNull() &
        col("cpf").isNotNull()
    )
    
    return df_transformed

def transform_medico_identidade(df: DataFrame) -> DataFrame:
    """
    Transformações medico_identidade:
    - Validar sk_medico (obrigatório)
    - Padronizar nome (initcap)
    - Limpar CRM (só números)
    """
    df_transformed = df.select(
        col("sk_medico").cast("int"),
        initcap(trim(col("nome_medico"))).alias("nome_medico"),
        regexp_replace(trim(col("crm")), r"[^\d]", "").alias("crm"),
        col("ingestion_timestamp"),
        col("is_pii")
    )
    
    # sk_medico é obrigatório
    df_transformed = df_transformed.filter(
        col("sk_medico").isNotNull()
    )
    
    return df_transformed

## Pipeline de Processamento

In [0]:
def process_pii_to_silver(table_name: str, transformation_func, key_column: str) -> dict:
    """
    Pipeline completo Bronze → Silver para PII
    """
    try:
        print(f"\n{'='*60}")
        print(f"Processando PII: {table_name}")
        print('='*60)
        
        # 1. Ler da Bronze/pii
        bronze_path = f"{BRONZE_PATH}/{table_name}"
        df_bronze = spark.read.format("delta").load(bronze_path)
        
        print(f"Lidos da Bronze: registros sensíveis")
        
        # 2. Aplicar transformações
        df_transformed = transformation_func(df_bronze)
        
        # 3. Remover duplicatas
        df_transformed = remove_duplicates(df_transformed, key_column)
        
        # 4. Adicionar metadados Silver
        df_transformed = add_silver_metadata(df_transformed)
        
        # 5. Escrever na Silver/pii com MERGE
        silver_path = f"{SILVER_PATH}/{table_name}"
        
        if DeltaTable.isDeltaTable(spark, silver_path):
            # Merge incremental
            delta_table = DeltaTable.forPath(spark, silver_path)
            
            delta_table.alias("target").merge(
                df_transformed.alias("source"),
                f"target.{key_column} = source.{key_column}"
            ).whenMatchedUpdateAll() \
             .whenNotMatchedInsertAll() \
             .execute()
            
            print(f"MERGE executado com sucesso")
            
        else:
            # Primeira carga
            df_transformed.write.format("delta") \
                .mode("overwrite") \
                .option("mergeSchema", "true") \
                .saveAsTable(f"healthcare_silver.{table_name}")
            
            print(f"Tabela criada com sucesso")
        
        return {
            "table": table_name,
            "status": "SUCCESS"
        }
        
    except Exception as e:
        print(f"ERRO: {str(e)}")
        return {
            "table": table_name,
            "status": "FAILED",
            "error": str(e)
        }

## Execução

In [0]:
# Configuração de tabelas PII
pii_config = [
    {"table": "paciente_identidade", "transformation": transform_paciente_identidade, "key": "sk_paciente"},
    {"table": "medico_identidade", "transformation": transform_medico_identidade, "key": "sk_medico"} 
]

# Processar todas as tabelas PII
results = []
for config in pii_config:
    result = process_pii_to_silver(
        table_name=config["table"],
        transformation_func=config["transformation"],
        key_column=config["key"]
    )
    results.append(result)

## Sumário de Execução

In [0]:
import pandas as pd

# Exibir resultados
results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("SUMÁRIO DE EXECUÇÃO - PII BRONZE TO SILVER")
print("="*80)
display(results_df)

# Estatísticas
total_tables = len(results)
success_count = len([r for r in results if r['status'] == 'SUCCESS'])
failed_count = len([r for r in results if r['status'] == 'FAILED'])

print(f"\nTabelas PII processadas: {success_count}/{total_tables}")

if failed_count > 0:
    print(f"ALERTA: {failed_count} tabelas PII falharam!")

print("\n" + "="*80)
print("ATENÇÃO: Dados sensíveis processados - Acesso restrito LGPD")
print("="*80)